# 패키지 설치 (python 3.8.10 기준)
아래 두 가지 방법 중 한 가지를 택하여 설치합니다.

## pip 사용
```pip install -r requirements.txt```

## conda 환경 사용 
```conda env create -f environment.yml```


In [ ]:
!pip install -r requirements.txt
#!conda env create -f environment.yml

In [ ]:
# 필요한 라이브러리 임포트
import os
import torch
from PIL import Image
import numpy as np
import requests
import matplotlib.pyplot as plt

# LAVIS (BLIP 모델용)
from lavis.models import load_model_and_preprocess

# Diffusers 및 Pix2Pix-Zero 유틸리티
from diffusers import DDIMScheduler
from src.utils.ddim_inv import DDIMInversion
from src.utils.scheduler import DDIMInverseScheduler
from src.utils.edit_pipeline import EditingPipeline
from src.utils.edit_directions import construct_direction

# 전역 변수 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16

## Direction for prompt embedding edit

In [ ]:
def construct_direction(task_name):
    # src2dst 에서 src 와 dst 분리 
    (src, dst) = task_name.split("2")
    # 미리 저장된 embedding 경로
    emb_dir = f"assets/embeddings_sd_1.4"
    # embs_a, emb_b에 각각 해당되는 embedding을 불러옴. 
    embs_a = torch.load(os.path.join(emb_dir, f"{src}.pt"), map_location=device)
    embs_b = torch.load(os.path.join(emb_dir, f"{dst}.pt"), map_location=device)
    return (embs_b.mean(0)-embs_a.mean(0)).unsqueeze(0)

## DDIM inversion

실제 이미지를 초기 노이즈(latent)로 변환하는 Inversion 과정을 함수로 정의합니다.

In [ ]:
def run_inversion(input_image: str,
                  results_folder: str,
                  num_ddim_steps: int,
                  model_path: str) -> (str, str, Image.Image, Image.Image):
    """
    주어진 이미지에 대해 DDIM Inversion을 수행합니다.

    Args:
        input_image (str): 입력 이미지의 경로
        results_folder (str): 결과가 저장될 폴더 경로
        num_ddim_steps (int): DDIM Inversion 스텝 수
        model_path (str): 사용할 Stable Diffusion 모델 경로

    Returns:
        tuple: (저장된 노이즈 경로, 저장된 프롬프트 경로, 원본 PIL 이미지, 복원된 PIL 이미지)
    """
    # --- 이미지 다운로드 및 폴더 생성 ---
    print("--- Inversion 시작 ---")
    os.makedirs(results_folder, exist_ok=True)
    os.makedirs(os.path.join(results_folder, "inversion"), exist_ok=True)
    os.makedirs(os.path.join(results_folder, "prompt"), exist_ok=True)

    # --- 1. 모델 로드 ---
    print("[1/4] BLIP 및 DDIM Inversion 모델을 로드합니다...")
    model_blip, vis_processors, _ = load_model_and_preprocess(name="blip_caption", model_type="base_coco", is_eval=True, device=device)
    pipe = DDIMInversion.from_pretrained(model_path, torch_dtype=torch_dtype).to(device)
    pipe.scheduler = DDIMInverseScheduler.from_config(pipe.scheduler.config)

    # --- 2. 캡션 생성 ---
    print("[2/4] 이미지 캡션을 생성합니다...")
    img = Image.open(input_image).resize((512, 512), Image.Resampling.LANCZOS)
    _image = vis_processors["eval"](img).unsqueeze(0).to(device)
    prompt_str = model_blip.generate({"image": _image})[0]
    print(f"     생성된 캡션: '{prompt_str}'")

    # --- 3. DDIM Inversion 실행 ---
    print("[3/4] DDIM Inversion을 실행하여 초기 노이즈를 찾습니다...")
    x_inv, _, x_dec_img = pipe(
        prompt_str,
        guidance_scale=1.0,
        num_inversion_steps=num_ddim_steps,
        img=img,
        torch_dtype=torch_dtype
    )

    # --- 4. 결과 저장 ---
    print("[4/4] Inversion 결과(노이즈)와 프롬프트를 저장합니다...")
    bname = os.path.basename(input_image).split(".")[0]
    inverted_latent_path = os.path.join(results_folder, f"inversion/{bname}.pt")
    prompt_path = os.path.join(results_folder, f"prompt/{bname}.txt")
    torch.save(x_inv[0], inverted_latent_path)
    with open(prompt_path, "w") as f:
        f.write(prompt_str)
    print(f"   - 노이즈 저장 경로: {inverted_latent_path}")
    print(f"   - 프롬프트 저장 경로: {prompt_path}")
    print("--- Inversion 완료 ---\n")

    return inverted_latent_path, prompt_path, img, x_dec_img[0]

## Edit image
Inversion으로 얻은 노이즈와 프롬프트를 사용해 이미지를 편집하는 과정을 함수로 정의합니다.

In [ ]:
def run_editing(inverted_latent_path: str,
                prompt_path: str,
                task_name: str,
                edit_results_folder: str,
                xa_guidance: float,
                negative_guidance_scale: float,
                num_ddim_steps: int,
                model_path: str) -> (Image.Image, Image.Image):
    """
    Inversion된 노이즈를 바탕으로 이미지를 편집합니다.

    Args:
        inverted_latent_path (str): 저장된 노이즈(.pt) 파일 경로
        prompt_path (str): 저장된 프롬프트(.txt) 파일 경로
        task_name (str): 편집 작업 이름 (예: 'cat2dog')
        edit_results_folder (str): 편집된 이미지가 저장될 폴더
        xa_guidance (float): 편집 방향 적용 강도
        negative_guidance_scale (float): Negative guidance 강도
        num_ddim_steps (int): DDIM 스텝 수
        model_path (str): 사용할 Stable Diffusion 모델 경로

    Returns:
        tuple: (복원된 PIL 이미지, 편집된 PIL 이미지)
    """
    # --- 폴더 생성 ---
    print("--- Editing 시작 ---")
    os.makedirs(os.path.join(edit_results_folder, "edit"), exist_ok=True)
    os.makedirs(os.path.join(edit_results_folder, "reconstruction"), exist_ok=True)

    # --- 1. 편집 파이프라인 로드 ---
    print("[1/4] 이미지 편집 파이프라인을 로드합니다...")
    pipe = EditingPipeline.from_pretrained(model_path, torch_dtype=torch_dtype).to(device)
    pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

    # --- 2. Inversion 결과 로드 ---
    print(f"[2/4] Inversion 결과를 로드합니다.")
    loaded_prompt_str = open(prompt_path).read().strip()
    x_in = torch.load(inverted_latent_path).unsqueeze(0).to(device)

    # --- 3. 편집 방향(Editing Direction) 설정 ---
    print(f"[3/4] '{task_name}' 작업에 대한 편집 방향을 설정합니다...")
    edit_dir = construct_direction(task_name)

    # --- 4. 편집 실행 ---
    print("[4/4] 이미지 편집을 실행합니다...")
    rec_pil, edit_pil = pipe(
        loaded_prompt_str,
        num_inference_steps=num_ddim_steps,
        x_in=x_in,
        edit_dir=edit_dir,
        guidance_amount=xa_guidance,
        guidance_scale=negative_guidance_scale,
        negative_prompt=loaded_prompt_str
    )

    # --- 결과 저장 ---
    bname = os.path.basename(inverted_latent_path).split(".")[0]
    edited_image_path = os.path.join(edit_results_folder, f"edit/{bname}_{task_name}.png")
    recon_image_path = os.path.join(edit_results_folder, f"reconstruction/{bname}.png")
    edit_pil[0].save(edited_image_path)
    rec_pil[0].save(recon_image_path)
    print(f"결과가 다음 경로에 저장되었습니다:\n   - 편집 이미지: {edited_image_path}")
    print("--- Editing 완료 ---\n")

    return rec_pil[0], edit_pil[0]

## Pipeline 실행 및 결과 확인
위에서 정의된 함수들을 호출하여 전체 과정을 실행합니다.

### Inversion 함수 호출

In [ ]:
# common parameters
model_path = 'CompVis/stable-diffusion-v1-4' 
num_ddim_steps = 50

# Inversion parameters
input_image = 'assets/test_images/cats/cat_1.png' 
inversion_folder = 'output/cat_test' 


inv_latent_path, inv_prompt_path, original_image, reconstructed_image = run_inversion(
    input_image=input_image,
    results_folder=inversion_folder,
    num_ddim_steps=num_ddim_steps,
    model_path=model_path
)


### Editing 함수 호출 및 시각화

In [ ]:
# Editing parameters
# (예: `cat2dog`, `horse2zebra`, `orange2apple`, `gold2wood`, `cat_statue`)
task_name = 'cat2dog'
editing_folder = 'output/test_cat' 
xa_guidance = 0.1
negative_guidance_scale = 5.0

# edit
final_reconstructed_image, edited_image = run_editing(
    inverted_latent_path=inv_latent_path,
    prompt_path=inv_prompt_path,
    task_name=task_name,
    edit_results_folder=editing_folder,
    xa_guidance=xa_guidance,
    negative_guidance_scale=negative_guidance_scale,
    num_ddim_steps=num_ddim_steps,
    model_path=model_path
)

In [ ]:
# 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(original_image)
axes[0].set_title("Original Image")
axes[0].axis('off')

axes[1].imshow(final_reconstructed_image)
axes[1].set_title("Reconstructed Image")
axes[1].axis('off')

axes[2].imshow(edited_image)
axes[2].set_title(f"Edited Image ('{task_name}')")
axes[2].axis('off')

plt.tight_layout()
plt.show()